## 0. Pipelines 

In [2]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [7]:
classifier('We are not very unhappy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.570195198059082}]

## 1 Under the hood

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [11]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# need to ensure the tokenizer and the model are compatible
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")
inputs

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Tokenizer's most important methods are encode and decode. Note the special [CLS] and [SEP] tokens added.

In [14]:
tokenizer.decode(inputs['input_ids'])

'[CLS] we are very happy to show you the [UNK] transformers library. [SEP]'

In [18]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
pt_batch

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996,   100,
         19081,  3075,  1012,   102],
        [  101,  2057,  3246,  2017,  2123,  1005,  1056,  5223,  2009,  1012,
           102,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

**Learning note:** The `attention_mask` is useful in two cases:
1. mask padding tokens
2. mask future tokens in autoregression models

### Using the model
Once your input has been preprocessed by the tokenizer, you can send it directly to the model. As we mentioned, it will contain all the relevant information the model needs. If you’re using a TensorFlow model, you can pass the dictionary keys directly to tensors, for a PyTorch model, you need to unpack the dictionary by adding `**`

In [24]:
pt_outputs = pt_model(**pt_batch)
pt_outputs

(tensor([[-4.0833,  4.3364],
         [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>),)

All 🤗 Transformers models (PyTorch or TensorFlow) return the activations of the model *before* the final activation
function (like SoftMax) since this final activation function is often fused with the loss.

In [26]:
import torch.nn.functional as F
pt_predictions = F.softmax(pt_outputs[0], dim=-1)
pt_predictions

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward>)

If you have labels, you can provide them to the model, it will return a tuple with the loss and the final activations.

In [28]:
import torch
pt_outputs = pt_model(**pt_batch, labels = torch.tensor([1, 0]))
pt_outputs

(tensor(0.3167, grad_fn=<NllLossBackward>),
 tensor([[-4.0833,  4.3364],
         [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>))

Once your model is fine-tuned, you can save it with its tokenizer in the following way:

In [ ]:
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

You can then load this model back using the `from_pretrained()` method by passing the directory name instead of the model name. One cool feature of 🤗 Transformers is that you can easily switch between PyTorch and TensorFlow: any model saved as before can be loaded back either in PyTorch or TensorFlow. If you are loading a saved PyTorch model in a TensorFlow model, use `from_pretrained()` like this:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = TFAutoModel.from_pretrained(save_directory, from_pt=True)

Lastly, you can also ask the model to return all hidden states and all attention weights if you need them:

In [29]:
pt_outputs = pt_model(**pt_batch, output_hidden_states=True, output_attentions=True)
all_hidden_states, all_attentions = pt_outputs[-2:]

The last layers of `pt_model`:
    
```
(pre_classifier): Linear(in_features=768, out_features=768, bias=True)
(classifier): Linear(in_features=768, out_features=2, bias=True)
(dropout): Dropout(p=0.2, inplace=False)
```

For something that only changes the head of the model (for instance, the number of labels), you can still use a pretrained model for the body. For instance, let’s define a classifier for 10 different labels using a pretrained body. We could create a configuration with all the default values and just change the number of labels, but more easily, you can directly pass any argument a configuration would take to the `from_pretrained()` method and it will update the default configuration with it:

In [37]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

The last layers of `model`:
    
```
(pre_classifier): Linear(in_features=768, out_features=768, bias=True)
(classifier): Linear(in_features=768, out_features=10, bias=True)
(dropout): Dropout(p=0.2, inplace=False)
```

## 2. Model Inputs

* Input IDs
* Attention masks
* Token Type IDs
* Labels

The first two we've already encountered.

### Token Type IDs

Some models’ purpose is to do sequence classification or question answering. These require two different sequences to be joined in a single “input_ids” entry, which usually is performed with the help of special tokens, such as the classifier (`[CLS]`) and separator (`[SEP]`) tokens. For example, the BERT model builds its two sequence input as such:

```
>>> # [CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
```

We can use our tokenizer to automatically generate such a sentence by passing the two sequences to tokenizer as two arguments (and not a list, like before) like this:

In [6]:
sequence_a = "HuggingFace is based in NYC"
sequence_b = "Where is HuggingFace based?"

In [5]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
encoded_dict = tokenizer(sequence_a, sequence_b)
decoded = tokenizer.decode(encoded_dict["input_ids"])
print(encoded_dict)
print(decoded)

{'input_ids': [0, 40710, 3923, 34892, 16, 716, 11, 14415, 2, 2, 13841, 16, 30581, 3923, 34892, 716, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
<s>HuggingFace is based in NYC</s></s>Where is HuggingFace based?</s>


In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
encoded_dict = tokenizer(sequence_a, sequence_b)
decoded = tokenizer.decode(encoded_dict["input_ids"])
print(decoded)

[CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]


The separater is enough for some models to understand where one sequence ends and where another begins. However, other models, such as BERT, also deploy token type IDs (also called segment IDs). They are represented as a binary mask identifying the two types of sequence in the model.

The tokenizer returns this mask as the “token_type_ids” entry:

In [9]:
encoded_dict['token_type_ids']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### Labels
The labels are an optional argument which can be passed in order for the model to compute the loss itself. These labels should be the expected prediction of the model: it will use the standard loss in order to compute the loss between its predictions and the expected value (the label).

These labels are different according to the model head, for example:

* For sequence classification models (e.g., `BertForSequenceClassification`), the model expects a tensor of dimension `(batch_size)` with each value of the batch corresponding to the expected label of the entire sequence.
* For token classification models (e.g., `BertForTokenClassification`), the model expects a tensor of dimension `(batch_size, seq_length)` with each value corresponding to the expected label of each individual token.
* For masked language modeling (e.g., `BertForMaskedLM`), the model expects a tensor of dimension `(batch_size, seq_length)` with each value corresponding to the expected label of each individual token: the labels being the token ID for the masked token, and values to be ignored for the rest (usually -100).
* For sequence to sequence tasks,(e.g., `BartForConditionalGeneration`, `MBartForConditionalGeneration`), the model expects a tensor of dimension `(batch_size, tgt_seq_length)` with each value corresponding to the target sequences associated with each input sequence. During training, both `BART` and `T5` will make the appropriate decoder_input_ids and decoder attention masks internally. They usually do not need to be supplied. This does not apply to models leveraging the Encoder-Decoder framework. See the documentation of each model for more information on each specific model’s labels.

The base models (e.g., `BertModel`) do not accept labels, as these are the base transformer models, simply outputting features.

### Decoder input IDs

This input is specific to encoder-decoder models, and contains the input IDs that will be fed to the decoder. These inputs should be used for sequence to sequence tasks, such as translation or summarization, and are usually built in a way specific to each model.

Most encoder-decoder models (`BART`, `T5`) create their decoder_input_ids on their own from the labels. In such models, passing the labels is the preferred way to handle training.

Please check each model’s docs to see how they handle these input IDs for sequence to sequence training.

## 3. Tokenizer Deep Dive
[[link]](https://huggingface.co/transformers/preprocessing.html)

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [2]:
encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)

{'input_ids': [101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [3]:
tokenizer.decode(encoded_input["input_ids"])

"[CLS] Hello, I'm a single sentence! [SEP]"

As you can see, the tokenizer automatically added some special tokens that the model expects. Not all models need special tokens; for instance, if we had used `gtp2-medium` instead of bert-base-cased to create our tokenizer, we would have seen the same sentence as the original one here. You can disable this behavior (which is only advised if you have added those special tokens yourself) by passing `add_special_tokens=False`

If you have several sentences you want to process, you can do this efficiently by sending them as a list to the tokenizer.

In [4]:
batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one"]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

{'input_ids': [[101, 8667, 146, 112, 182, 170, 1423, 5650, 102], [101, 1262, 1330, 5650, 102], [101, 1262, 1103, 1304, 1304, 1314, 1141, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


If the purpose of sending several sentences at a time to the tokenizer is to build a batch to feed the model, you will probably want:

* To pad each sentence to the maximum length there is in your batch.
* To truncate each sentence to the maximum length the model can accept (if applicable).
* To return tensors.

You can do all of this by using the following options when feeding your list of sentences to the tokenizer:

In [9]:
tokenizer(batch_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

{'input_ids': tensor([[ 101, 8667,  146,  112,  182,  170, 1423, 5650,  102],
        [ 101, 1262, 1330, 5650,  102,    0,    0,    0,    0],
        [ 101, 1262, 1103, 1304, 1304, 1314, 1141,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0]])}

### Preprocessing pairs of sentences

Sometimes you need to feed a pair of sentences to your model. For instance, if you want to classify if two sentences in a pair are similar, or for question-answering models, which take a context and a question. For BERT models, the input is then represented like this: `[CLS]` Sequence A `[SEP]` Sequence B `[SEP]`

You can encode a pair of sentences in the format expected by your model by supplying the two sentences as two arguments (not a list since a list of two sentences will be interpreted as a batch of two single sentences, as we saw before). This will once again return a dict string to list of ints:

In [11]:
encoded_input = tokenizer("How old are you?", "I'm 6 years old")
print(encoded_input)
print(tokenizer.decode(encoded_input['input_ids']))

{'input_ids': [101, 1731, 1385, 1132, 1128, 136, 102, 146, 112, 182, 127, 1201, 1385, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] How old are you? [SEP] I'm 6 years old [SEP]


This shows us what the token_type_ids are for: they indicate to the model which part of the inputs correspond to the first sentence and which part corresponds to the second sentence. Note that `token_type_ids` are not required or handled by all models. By default, a tokenizer will only return the inputs that its associated model expects. You can force the return (or the non-return) of any of those special arguments by using `return_input_ids` or `return_token_type_ids`.



If you have a list of pairs of sequences you want to process, you should feed them as two lists to your tokenizer: the list of first sentences and the list of second sentences:

In [12]:
batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one"]
batch_of_second_sentences = ["I'm a sentence that goes with the first sentence",
                              "And I should be encoded with the second sentence",
                              "And I go with the very last one"]
encoded_inputs = tokenizer(batch_sentences, batch_of_second_sentences)

### Pre-tokenized inputs

The tokenizer also accept pre-tokenized inputs. This is particularly useful when you want to compute labels and extract predictions in named entity recognition (NER) or part-of-speech tagging (POS tagging).

If you want to use pre-tokenized inputs, just set `is_split_into_words=True` when passing your inputs to the tokenizer. For instance, we have:

In [13]:
encoded_input = tokenizer(["Hello", "I'm", "a", "single", "sentence"], is_split_into_words=True)
print(encoded_input)

{'input_ids': [101, 8667, 146, 112, 182, 170, 1423, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
tokenizer.decode(encoded_input['input_ids'], clean_up_tokenization_spaces=False)

"[CLS] Hello I ' m a single sentence [SEP]"

**Note:** the output of the tokenizer can be different from the pre-tokenized input.